In [1]:
%ls

Datasets/                                 nltk_data/
GermanT5-RP-Mod/                          ray_results/
RP-Mod/                                   results/
RP-Mod-GermanT5-oscar-german-small-el32/  t5_base_imdb_sentiment/
T5FineTuner.py                            t5_german_small_rp_mod/
aclImdb/                                  t5_german_small_rp_mod_2/
aclImdb_v1.tar.gz                         t5_german_small_rp_mod_3/
arguments_test_dir/                       t5_imdb_sentiment/
create_t5_embeddings.ipynb                wandb/
lightning_logs/


In [22]:
from T5FineTuner import T5FineTuner, RPDataset
from utils import get_folds
import torch
import argparse
from transformers import T5Tokenizer
from torch.utils.data import Dataset, DataLoader

DATASET = "RP-Crowd-3"
MODEL_NAME_OR_PATH = "GermanT5/t5-efficient-oscar-german-small-el32"
WANDB_PROJECT_NAME = "rp-crowd-3-folds-t5-efficient-small-el32"
OUTPUT_DIR = "./GermanT5-RP-Mod/t5-efficient-oscar-german-small-el32/"
SOURCE = f"./Datasets/{DATASET}-folds.csv"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME_OR_PATH)
checkpoint_path = "./GermanT5-RP-Mod/t5-efficient-oscar-german-small-el32/lr-0.0004-wd-0.1/epoch=3-val_accuracy=0.74-val_loss=0.28.ckpt"
train_inputs, train_targets, val_inputs, val_targets = get_folds(SOURCE)

train_dataset = RPDataset(tokenizer, train_inputs, train_targets)
valid_dataset = RPDataset(tokenizer, val_inputs, val_targets)

checkpoint = torch.load(checkpoint_path)
hparams = checkpoint["hyper_parameters"]
hparams["train_dataset"] = train_dataset
hparams["val_dataset"] = valid_dataset
new_args = argparse.Namespace(**hparams)

new_model = T5FineTuner.load_from_checkpoint(checkpoint_path, hparams=new_args)


In [20]:
val_inputs[0], val_targets[0]

('classification Tja da weiß man wie die Intelligenz der Mehrheit der Türken einzuschätzen ist',
 'problematisch')

In [25]:
from sklearn import metrics
val_dataloader = DataLoader(valid_dataset, batch_size=64)

# for batch in val_dataloader:
batch = next(iter(val_dataloader))
labels = batch["target_ids"]
outs = new_model.model.generate(input_ids=batch["source_ids"], 
                            attention_mask=batch["source_mask"], 
                            max_length=2)

dec = [tokenizer.decode(ids) for ids in outs]
target = [tokenizer.decode(label) for label in labels]

new_outputs = [s[6:] for s in dec]
new_targets = [s[:-4] for s in target]

f1 = metrics.f1_score(new_targets, new_outputs, pos_label="problematisch")
print(f1)

0.7419354838709677
